In [0]:
from delta.tables import *

from pyspark.sql.types import * 

from pyspark.sql.functions import *

In [0]:
DeltaTable.createOrReplace(spark)\
          .tableName('employee_demo')\
          .addColumn('emp_id',IntegerType(),False)\
          .addColumn('emp_name',StringType(),False)\
          .addColumn('gender',StringType(),False)\
          .addColumn('sal',IntegerType(),True)\
          .addColumn('dept',StringType(),True)\
          .property("description","for practise purpose")\
          .location("/FilseStore/tables/delta/practice_delta")\
          .execute()

Out[6]: <delta.tables.DeltaTable at 0x7ffada0cee80>

In [0]:
%sql 

select * from employee_demo

emp_id,emp_name,gender,sal,dept


In [0]:
%sql

insert into employee_demo values(1,"varun","M",13,"it");
insert into employee_demo values(2,"sahithi","F",15,"hr");
insert into employee_demo values(3,"kusuma","F",11,"sales");



num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql 

select * from employee_demo

emp_id,emp_name,gender,sal,dept
3,kusuma,F,11,sales
2,sahithi,F,15,hr
1,varun,M,13,it


#Approach_1

In [0]:
Delta_Instance_1=DeltaTable.forPath(spark,'/FilseStore/tables/delta/practice_delta')

In [0]:
display(Delta_Instance_1.toDF())

emp_id,emp_name,gender,sal,dept
3,kusuma,F,11,sales
2,sahithi,F,15,hr
1,varun,M,13,it


In [0]:
%sql 

delete from employee_demo where emp_id=2

num_affected_rows
1


In [0]:
display(Delta_Instance_1.toDF())

emp_id,emp_name,gender,sal,dept
3,kusuma,F,11,sales
1,varun,M,13,it


In [0]:
#deleting on delta instance

Delta_Instance_1.delete("emp_id=1")

In [0]:
display(Delta_Instance_1.toDF())

emp_id,emp_name,gender,sal,dept
3,kusuma,F,11,sales


#Approach_2: via name

In [0]:
Delta_Instance_2=DeltaTable.forName(spark,'employee_demo')

In [0]:
display(Delta_Instance_2.toDF())

emp_id,emp_name,gender,sal,dept
3,kusuma,F,11,sales


In [0]:
%sql 

describe history employee_demo

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
5,2024-06-05T16:51:23.000+0000,8432478468657958,paruchuri7781@gmail.com,DELETE,"Map(predicate -> [""(emp_id#2330 = 1)""])",null,List(3115176791291459),0605-155657-39o35iz8,4,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 1498, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2746, numDeletedRows -> 1, scanTimeMs -> 1117, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 1628)",null,Databricks-Runtime/12.2.x-scala2.12
4,2024-06-05T16:49:23.000+0000,8432478468657958,paruchuri7781@gmail.com,DELETE,"Map(predicate -> [""(emp_id#2463 = 2)""])",null,List(3115176791291459),0605-155657-39o35iz8,3,WriteSerializable,false,"Map(numRemovedFiles -> 1, numRemovedBytes -> 1512, numCopiedRows -> 0, numDeletionVectorsAdded -> 0, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 4227, numDeletedRows -> 1, scanTimeMs -> 2447, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 1760)",null,Databricks-Runtime/12.2.x-scala2.12
3,2024-06-05T16:17:20.000+0000,8432478468657958,paruchuri7781@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(3115176791291459),0605-155657-39o35iz8,2,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1526)",null,Databricks-Runtime/12.2.x-scala2.12
2,2024-06-05T16:17:13.000+0000,8432478468657958,paruchuri7781@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(3115176791291459),0605-155657-39o35iz8,1,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1512)",null,Databricks-Runtime/12.2.x-scala2.12
1,2024-06-05T16:17:06.000+0000,8432478468657958,paruchuri7781@gmail.com,WRITE,"Map(mode -> Append, partitionBy -> [])",null,List(3115176791291459),0605-155657-39o35iz8,0,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1498)",null,Databricks-Runtime/12.2.x-scala2.12
0,2024-06-05T16:13:03.000+0000,8432478468657958,paruchuri7781@gmail.com,CREATE OR REPLACE TABLE,"Map(isManaged -> false, description -> null, partitionBy -> [], properties -> {""description"":""for practise purpose""})",null,List(3115176791291459),0605-155657-39o35iz8,null,WriteSerializable,true,Map(),null,Databricks-Runtime/12.2.x-scala2.12


In [0]:
emp_DT_history=Delta_Instance_2.history()
emp_DT_history.show()

+-------+-------------------+----------------+--------------------+--------------------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|          userId|            userName|           operation| operationParameters| job|          notebook|           clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+----------------+--------------------+--------------------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|      5|2024-06-05 16:51:23|8432478468657958|paruchuri7781@gma...|              DELETE|{predicate -> ["(...|null|{3115176791291459}|0605-155657-39o35iz8|          4|WriteSerializable|        false|{numRemovedFiles ...|        null|Databr

In [0]:
emp_DT_history.select(col('version'),col('operation'))\
              .show()

+-------+--------------------+
|version|           operation|
+-------+--------------------+
|      5|              DELETE|
|      4|              DELETE|
|      3|               WRITE|
|      2|               WRITE|
|      1|               WRITE|
|      0|CREATE OR REPLACE...|
+-------+--------------------+



In [0]:
emp_DT_history.filter(col('version')==4).show()

+-------+-------------------+----------------+--------------------+---------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|          userId|            userName|operation| operationParameters| job|          notebook|           clusterId|readVersion|   isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+----------------+--------------------+---------+--------------------+----+------------------+--------------------+-----------+-----------------+-------------+--------------------+------------+--------------------+
|      4|2024-06-05 16:49:23|8432478468657958|paruchuri7781@gma...|   DELETE|{predicate -> ["(...|null|{3115176791291459}|0605-155657-39o35iz8|          3|WriteSerializable|        false|{numRemovedFiles ...|        null|Databricks-Runtim...|
+-------+-------------------

---------------------------------------------------------------------------
IllegalArgumentException                  Traceback (most recent call last)
File <command-445368849526868>:1
----> 1 Delta_Instance_2.history(4).toDF()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:4949, in DataFrame.toDF(self, *cols)
   4944     if not isinstance(col, str):
   4945         raise PySparkTypeError(
   4946             error_class="NOT_LIST_OF_STR",
   4947             message_parameters={"arg_name": "cols", "arg_type": type(col).__name__},
   4948         )
-> 4949 jdf = self._jdf.toDF(self._jseq(cols))
   4950 return D

altering schema

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-3549299761892539>:1
----> 1 Delta_Instance_2.cloneAtTimestamp('2024-06-05 16:49:23',employee_demo)

NameError: name 'employee_demo' is not defined